# ETL

In [1]:
# importing libraries
import pandas as pd
import pyodbc
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
# loading the csv file in which contains the loans data
df = pd.read_csv('financial_loan.csv')
df.head()

,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,loan_status,next_payment_date,member_id,purpose,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment
0,1077430,GA,INDIVIDUAL,< 1 year,Ryder,C,RENT,11-02-2021,13-09-2021,13-04-2021,Charged Off,13-05-2021,1314167,car,C4,60 months,Source Verified,30000.0,0.0100,59.83,0.1527,2500,4,1009
1,1072053,CA,INDIVIDUAL,9 years,MKC Accounting,E,RENT,01-01-2021,14-12-2021,15-01-2021,Fully Paid,15-02-2021,1288686,car,E1,36 months,Source Verified,48000.0,0.0535,109.43,0.1864,3000,4,3939
2,1069243,CA,INDIVIDUAL,4 years,Chemat Technology Inc,C,RENT,05-01-2021,12-12-2021,09-01-2021,Charged Off,09-02-2021,1304116,car,C5,36 months,Not Verified,50000.0,0.2088,421.65,0.1596,12000,11,3522
3,1041756,TX,INDIVIDUAL,< 1 year,barnes distribution,B,MORTGAGE,25-02-2021,12-12-2021,12-03-2021,Fully Paid,12-04-2021,1272024,car,B2,60 months,Source Verified,42000.0,0.0540,97.06,0.1065,4500,9,4911
4,1068350,IL,INDIVIDUAL,10+ years,J&J Steel Inc,A,MORTGAGE,01-01-2021,14-12-2021,15-01-2021,Fully Paid,15-02-2021,1302971,car,A1,36 months,Verified,83000.0,0.0231,106.53,0.0603,3500,28,3835


Each row seem to represent one individual loan.

In [3]:
# seeing data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38576 entries, 0 to 38575
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     38576 non-null  int64  
 1   address_state          38576 non-null  object 
 2   application_type       38576 non-null  object 
 3   emp_length             38576 non-null  object 
 4   emp_title              37138 non-null  object 
 5   grade                  38576 non-null  object 
 6   home_ownership         38576 non-null  object 
 7   issue_date             38576 non-null  object 
 8   last_credit_pull_date  38576 non-null  object 
 9   last_payment_date      38576 non-null  object 
 10  loan_status            38576 non-null  object 
 11  next_payment_date      38576 non-null  object 
 12  member_id              38576 non-null  int64  
 13  purpose                38576 non-null  object 
 14  sub_grade              38576 non-null  object 
 15  te

I've spotted some null values. Let's take a closer look.

In [4]:

for col in df.columns:
    n_null = df[col].isnull().sum()
    if n_null > 0:
        print(f'{col}: {n_null} null values')

emp_title: 1438 null values


Employee title got some null values. We have to analyze if there are any logic on it.

In [5]:
# analyzing loan data for clients with blank occupations
df[df['emp_title'].isnull()]

,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,loan_status,next_payment_date,member_id,purpose,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment
19,1058060,MD,INDIVIDUAL,10+ years,NaN,D,OWN,02-02-2021,16-05-2021,15-02-2021,Fully Paid,15-03-2021,1289636,car,D1,36 months,Not Verified,40000.0,0.0957,176.51,0.1629,5000,13,6361
21,207910,FL,INDIVIDUAL,< 1 year,NaN,A,MORTGAGE,08-01-2021,16-05-2021,10-02-2021,Charged Off,10-03-2021,183496,car,A2,36 months,Not Verified,120000.0,0.0767,69.14,0.0743,2225,20,2508
31,744344,WA,INDIVIDUAL,6 years,NaN,B,MORTGAGE,11-05-2021,16-05-2021,13-01-2021,Charged Off,13-02-2021,942813,car,B4,36 months,Not Verified,48000.0,0.2357,197.83,0.1149,6000,27,3950
239,494363,NM,INDIVIDUAL,1 year,NaN,B,MORTGAGE,10-03-2021,11-08-2021,11-09-2021,Fully Paid,11-10-2021,632851,car,B4,36 months,Not Verified,45000.0,0.1125,108.03,0.1099,3300,13,3712
245,781298,TX,INDIVIDUAL,9 years,NaN,B,MORTGAGE,11-06-2021,16-05-2021,14-06-2021,Fully Paid,14-07-2021,984137,car,B2,36 months,Not Verified,70000.0,0.2309,278.85,0.1114,8500,19,10038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38305,617250,CO,INDIVIDUAL,5 years,NaN,B,RENT,10-11-2021,13-10-2021,13-10-2021,Fully Paid,13-11-2021,791380,wedding,B2,36 months,Verified,48000.0,0.2110,348.69,0.0925,18000,25,12545
38311,543406,NY,INDIVIDUAL,1 year,NaN,B,RENT,10-07-2021,12-05-2021,12-04-2021,Fully Paid,12-05-2021,701084,wedding,B2,36 months,Verified,96000.0,0.0084,130.49,0.1075,4000,14,4564
38314,375077,NY,INDIVIDUAL,3 years,NaN,B,RENT,09-02-2021,10-12-2021,10-12-2021,Fully Paid,10-01-2022,396883,wedding,B4,36 months,Verified,73500.0,0.0405,497.46,0.1189,15000,9,16833
38326,781758,CA,INDIVIDUAL,5 years,NaN,C,RENT,11-06-2021,16-04-2021,14-06-2021,Fully Paid,14-07-2021,984673,wedding,C4,36 months,Verified,60000.0,0.2024,104.91,0.1562,3000,23,3777


Though these clients got record regarding time of work, it's not possible to determine whether one is employed or not. So, I'll be filling the null values with "No information".

In [6]:
# replacing null values
df['emp_title'] = df['emp_title'].fillna('No information')
print(f'Number of null values in emp_title column: {df['emp_title'].isnull().sum()}')

Number of null values in emp_title column: 0


In [7]:
# verifying data redundancy
for col in df.columns:
    print(f'{col}: {df[col].nunique()} values')

id: 38576 values
address_state: 50 values
application_type: 1 values
emp_length: 11 values
emp_title: 28526 values
grade: 7 values
home_ownership: 5 values
issue_date: 65 values
last_credit_pull_date: 107 values
last_payment_date: 102 values
loan_status: 3 values
next_payment_date: 102 values
member_id: 38576 values
purpose: 14 values
sub_grade: 35 values
term: 2 values
verification_status: 3 values
annual_income: 5096 values
dti: 2863 values
installment: 15132 values
int_rate: 371 values
loan_amount: 880 values
total_acc: 82 values
total_payment: 19525 values


In [8]:
df['application_type'].unique()

array(['INDIVIDUAL'], dtype=object)

It seems like the application type have only one information, so we'll be discarding it.

In [9]:
df.drop('application_type', axis=1, inplace=True)

In [10]:
# taking a look at the two ID columns
df[['id', 'member_id']]

,id,member_id
0,1077430,1314167
1,1072053,1288686
2,1069243,1304116
3,1041756,1272024
4,1068350,1302971
...,...,...
38571,803452,992381
38572,970377,1192039
38573,875376,1089898
38574,972997,1194971


As ID and member ID columns seem to be different things, I'll be keeping them in the table.

In [11]:
df['term'].unique()

array([' 60 months', ' 36 months'], dtype=object)

The term column can be better stored in two numbers.

In [12]:
df['term'] = df['term'].str.extract('(\d+)').astype(int)
df['term'].unique()

array([60, 36])

### **Now, I'll load the DataFrame into a table in SQL Server.**

In [14]:
# connecting the database
conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=DESKTOP-JF730R8\SQLEXPRESS;'
    'Database=PortfolioData;'
    'Trusted_Connection=yes'
)

cursor = conn.cursor()

# creating the table
create_table_query = '''
CREATE TABLE BankLoans (
    id BIGINT
    , address_state NVARCHAR(5)
    , emp_lenght NVARCHAR(10)
    , emp_title NVARCHAR(255)
    , grade NVARCHAR(1)
    , home_ownership NVARCHAR(10)
    , issue_date DATE
    , last_credit_pull_date DATE
    , last_payment_date DATE
    , loan_status NVARCHAR(15)
    , next_payment_date DATE
    , member_id BIGINT
    , purpose NVARCHAR(30)
    , sub_grade NVARCHAR(2)
    , term_months INT
    , verification_status NVARCHAR(20)
    , annual_income FLOAT
    , dti FLOAT
    , installment FLOAT
    , int_rate FLOAT
    , loan_amount FLOAT
    , total_acc INT
    , total_payment FLOAT
)
'''

cursor.execute(create_table_query)
conn.commit()

# inserting the data into the table
for index, row in df.iterrows():
    cursor.execute('''
    INSERT INTO BankLoans (
        id
        , address_state
        , emp_lenght
        , emp_title
        , grade
        , home_ownership
        , issue_date
        , last_credit_pull_date
        , last_payment_date
        , loan_status
        , next_payment_date
        , member_id
        , purpose
        , sub_grade
        , term_months
        , verification_status
        , annual_income
        , dti
        , installment
        , int_rate
        , loan_amount
        , total_acc
        , total_payment
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
    row['id'],
    row['address_state'], 
    row['emp_length'], 
    row['emp_title'], 
    row['grade'], 
    row['home_ownership'], 
    row['issue_date'], 
    row['last_credit_pull_date'], 
    row['last_payment_date'], 
    row['loan_status'], 
    row['next_payment_date'], 
    row['member_id'], 
    row['purpose'], 
    row['sub_grade'], 
    row['term'], 
    row['verification_status'], 
    row['annual_income'], 
    row['dti'], 
    row['installment'], 
    row['int_rate'], 
    row['loan_amount'], 
    row['total_acc'], 
    row['total_payment']
    )
conn.commit()

print('The data was successfully inserted into SQL Server')
conn.close()

The data was successfully inserted into SQL Server
